# Laboratorio #4

**Esteban Zambrano - 22119**<br>
**Andrés Ortega - 22305**<br>
**Diego García - 22404**

In [6]:
#!pip install openeo
#!pip install rasterio

In [15]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import openeo
import glob
import os

In [8]:
connection = openeo.connect("https://openeo.dataspace.copernicus.eu").authenticate_oidc()

Authenticated using refresh token.


In [9]:
#Areas de interes
lago_atitlan = {
    "west": -91.349,
    "east": -91.0702,
    "south": 14.5971,
    "north": 14.7648
}
lago_amatitlan = {
    "west": -90.66,
    "east": -90.58,
    "south": 14.43,
    "north": 14.51
}

bands_needed = ["B02","B03","B04","B05","B07","B08","B8A","B11","B12"]

In [10]:
# Código de https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/cyanobacteria_chla_ndci_l1c/#representative-images pasado a python

# Water body detection - credit Mohor Gartner
MNDWI_threshold = 0.42
NDWI_threshold = 0.4
filter_UABS = True
filter_SSI = False

def wbi(r, g, b, nir, swir1, swir2):
    # water surface
    ws = 0
    try:
        # Calc indices
        ndvi = (nir - r) / (nir + r)
        mndwi = (g - swir1) / (g + swir1)
        ndwi = (g - nir) / (g + nir)
        ndwi_leaves = (nir - swir1) / (nir + swir1)
        aweish = b + 2.5 * g - 1.5 * (nir + swir1) - 0.25 * swir2
        aweinsh = 4 * (g - swir1) - (0.25 * nir + 2.75 * swir1)

        dbsi = ((swir1 - g) / (swir1 + g)) - ndvi
        wii = np.power(nir, 2) / r
        wri = (g + r) / (nir + swir1)
        puwi = 5.83 * g - 6.57 * r - 30.32 * nir + 2.25
        uwi = (g - 1.1 * r - 5.2 * nir + 0.4) / np.abs(g - 1.1 * r - 5.2 * nir)
        usi = 0.25 * (g / r) - 0.57 * (nir / g) - 0.83 * (b / g) + 1

        if (mndwi > MNDWI_threshold or ndwi > NDWI_threshold or
            aweinsh > 0.1879 or aweish > 0.1112 or
            ndvi < -0.2 or ndwi_leaves > 1):
            ws = 1

        if filter_UABS and ws == 1:
            if (aweinsh <= -0.03) or (dbsi > 0):
                ws = 0

    except Exception:
        ws = 0
    return ws

# Floating vegetation
def FAI(a, b, c):
    return b - a - (c - a) * (783 - 665) / (865 - 665)

# Chlorophyll-a
def NDCI(a, b):
    return (b - a) / (b + a)

def classify_pixel(B02, B03, B04, B05, B07, B08, B8A, B11, B12):
    water = wbi(B04, B03, B02, B08, B11, B12)
    FAIv = FAI(B04, B07, B8A)
    NDCIv = NDCI(B04, B05)
    chl = 826.57 * NDCIv**3 - 176.43 * NDCIv**2 + 19 * NDCIv + 4.071

    trueColor = [3 * B04, 3 * B03, 3 * B02]

    #  Render colour map
    if water == 0:
        return trueColor
    elif FAIv > 0.08:
        return [233/255, 72/255, 21/255]
    elif chl < 0.5:
        return [0, 0, 1.0]
    elif chl < 1:
        return [0, 0, 1.0]
    elif chl < 2.5:
        return [0, 59/255, 1]
    elif chl < 3.5:
        return [0, 98/255, 1]
    elif chl < 5:
        return [15/255, 113/255, 141/255]
    elif chl < 7:
        return [14/255, 141/255, 120/255]
    elif chl < 8:
        return [13/255, 141/255, 103/255]
    elif chl < 10:
        return [30/255, 226/255, 28/255]
    elif chl < 14:
        return [42/255, 226/255, 28/255]
    elif chl < 18:
        return [68/255, 226/255, 28/255]
    elif chl < 20:
        return [68/255, 226/255, 28/255]
    elif chl < 24:
        return [134/255, 247/255, 0]
    elif chl < 28:
        return [140/255, 247/255, 0]
    elif chl < 30:
        return [205/255, 237/255, 0]
    elif chl < 38:
        return [208/255, 240/255, 0]
    elif chl < 45:
        return [208/255, 240/255, 0]
    elif chl < 50:
        return [251/255, 210/255, 3/255]
    elif chl < 75:
        return [248/255, 207/255, 2/255]
    elif chl < 90:
        return [134/255, 247/255, 0]
    elif chl < 100:
        return [245/255, 164/255, 9/255]
    elif chl < 150:
        return [240/255, 159/255, 8/255]
    elif chl < 250:
        return [237/255, 157/255, 7/255]
    elif chl < 300:
        return [239/255, 118/255, 15/255]
    elif chl < 350:
        return [239/255, 101/255, 15/255]
    elif chl < 450:
        return [239/255, 100/255, 14/255]
    elif chl < 500:
        return [233/255, 72/255, 21/255]
    else:
        return [233/255, 72/255, 21/255]

In [11]:
def download_lake(spatial_extent, output_path):
    cube = connection.load_collection(
        "SENTINEL2_L2A",
        spatial_extent=spatial_extent,
        temporal_extent=["2025-02-01", "2025-07-31"],
        bands=bands_needed,
        max_cloud_cover=20
    )
    
    # Guardar resultado temporal en GeoTIFF
    temp_result = cube.save_result(format="GTIFF")
    job = connection.create_job(temp_result)
    job.start_and_wait()
    job.download_results(output_path)


def process_image(input_tif, output_tif):
    with rasterio.open(input_tif) as src:
        bands_data = src.read()
        transform = src.transform
        crs = src.crs
    
    # Extraer bandas por nombre
    B02, B03, B04, B05, B07, B08, B8A, B11, B12 = bands_data
    
    # Salida con 3 canales RGB
    rgb = np.zeros((3, B02.shape[0], B02.shape[1]), dtype=np.float32)
    
    for i in range(B02.shape[0]):
        for j in range(B02.shape[1]):
            rgb_pixel = classify_pixel(
                B02[i, j], B03[i, j], B04[i, j], B05[i, j],
                B07[i, j], B08[i, j], B8A[i, j], B11[i, j], B12[i, j]
            )
            rgb[:, i, j] = rgb_pixel
    
    # Guardar resultado
    with rasterio.open(
        output_tif,
        "w",
        driver="GTiff",
        height=rgb.shape[1],
        width=rgb.shape[2],
        count=3,
        dtype=rgb.dtype,
        crs=crs,
        transform=transform
    ) as dst:
        dst.write(rgb[0], 1)
        dst.write(rgb[1], 2)
        dst.write(rgb[2], 3)

In [13]:
# Descargar imagenes de los lagos

# download_lake(lago_atitlan, "../data/imgsAti")
download_lake(lago_amatitlan, "../data/imgsAmati")

0:00:00 Job 'j-2508140616094aeab545825e9c041a06': send 'start'
0:00:13 Job 'j-2508140616094aeab545825e9c041a06': created (progress 0%)
0:00:18 Job 'j-2508140616094aeab545825e9c041a06': created (progress 0%)
0:00:25 Job 'j-2508140616094aeab545825e9c041a06': created (progress 0%)
0:00:33 Job 'j-2508140616094aeab545825e9c041a06': created (progress 0%)
0:00:43 Job 'j-2508140616094aeab545825e9c041a06': created (progress 0%)
0:00:55 Job 'j-2508140616094aeab545825e9c041a06': created (progress 0%)
0:01:11 Job 'j-2508140616094aeab545825e9c041a06': running (progress N/A)
0:01:30 Job 'j-2508140616094aeab545825e9c041a06': running (progress N/A)
0:01:55 Job 'j-2508140616094aeab545825e9c041a06': running (progress N/A)
0:02:26 Job 'j-2508140616094aeab545825e9c041a06': running (progress N/A)
0:03:03 Job 'j-2508140616094aeab545825e9c041a06': running (progress N/A)
0:03:50 Job 'j-2508140616094aeab545825e9c041a06': running (progress N/A)
0:04:48 Job 'j-2508140616094aeab545825e9c041a06': finished (progres

C:\Users\dgv31\AppData\Local\Temp\ipykernel_26224\4160958946.py:14: UserDeprecationWarning: Call to deprecated method download_results. (Instead use `BatchJob.get_results` and the more flexible download functionality of `JobResults`) -- Deprecated since version 0.4.10.
  job.download_results(output_path)


In [16]:
# Procesar las imágenes descargadas según el script de sentinel-hub

file_ati = glob.glob(os.path.join("../data/imgsAti/", "openEO_*.tif"))[0]
file_amati = glob.glob(os.path.join("../data/imgsAmati/", "openEO_*.tif"))[0]

print("Archivo encontrado Ati:", file_ati)
print("Archivo encontrado Amati:", file_amati)

# Procesar usando tus funciones
for tif_file in glob.glob(os.path.join("../data/imgsAti/", "openEO_*.tif")):
    date = os.path.basename(tif_file).split("_")[1]  # Extrae "2025-03-07"
    output_name = f"../data/cyanobacAti/chl_Atitlan_{date}.tif"
    process_image(tif_file, output_name)

for tif_file in glob.glob(os.path.join("../data/imgsAmati/", "openEO_*.tif")):
    date = os.path.basename(tif_file).split("_")[1]  # Extrae "2025-03-07"
    output_name = f"../data/cyanobacAmati/chl_Amatitlan_{date}.tif"
    process_image(tif_file, output_name)

Archivo encontrado Ati: ../data/imgsAti\openEO_2025-02-07Z.tif
Archivo encontrado Amati: ../data/imgsAmati\openEO_2025-02-02Z.tif


C:\Users\dgv31\AppData\Local\Temp\ipykernel_26224\1061942483.py:43: RuntimeWarning: overflow encountered in scalar multiply
  return b - a - (c - a) * (783 - 665) / (865 - 665)
C:\Users\dgv31\AppData\Local\Temp\ipykernel_26224\1061942483.py:22: RuntimeWarning: divide by zero encountered in scalar divide
  wii = np.power(nir, 2) / r
C:\Users\dgv31\AppData\Local\Temp\ipykernel_26224\1061942483.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  usi = 0.25 * (g / r) - 0.57 * (nir / g) - 0.83 * (b / g) + 1
C:\Users\dgv31\AppData\Local\Temp\ipykernel_26224\1061942483.py:26: RuntimeWarning: invalid value encountered in scalar subtract
  usi = 0.25 * (g / r) - 0.57 * (nir / g) - 0.83 * (b / g) + 1
C:\Users\dgv31\AppData\Local\Temp\ipykernel_26224\1061942483.py:47: RuntimeWarning: divide by zero encountered in scalar divide
  return (b - a) / (b + a)
C:\Users\dgv31\AppData\Local\Temp\ipykernel_26224\1061942483.py:53: RuntimeWarning: invalid value encountered in scalar subtract